In [53]:
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import sys

In [54]:
def make_dict(text):
    text = text.replace('null',"'null'")
    return(eval(text[text.find('{'):text.find('}')+1]))

def dict_extend(hh_ls, driver, typeouter):
    req = driver.page_source
    soup = bs(req, 'html.parser')
    
    for i in range(0,10):
        try:
            hh_ls.append(make_dict(str(soup.find_all('td', 'al')[2*i])))
        except:
            break
    for i in range(0,10):
        try:
            driver.find_element_by_xpath("//*[@id='schTbody']/tr["+str(i+1)+"]/td[3]/a").send_keys(Keys.ENTER)
            time.sleep(2)
            driver.switch_to.window(driver.window_handles[1])
            req = driver.page_source
            soup = bs(req, 'html.parser')
            num = len(soup.find('tbody',{'id':'tblByType'}).find_all('td','ac')) #매입임대는 다 ar임.
            typeinner = []
            overlap = [text.text for text in soup.find_all('td')][:10]
            for j in range(0,num+1):  
                if len(typeinner)!=0: typeouter.append(typeinner)
                typeinner=[]
                typeinner.extend(overlap)
                for k in range(0,5):
                    typeinner.append(soup.find('tbody',{'id':'tblByType'}).find_all('td',
                    'ar')[k+5*j-5].text.replace('\xa0',''))
            time.sleep(2)
            driver.switch_to.window(driver.window_handles[0])
        except:
            break
    
def click_wait(wait,xpath):
    wait.until(EC.visibility_of_element_located((By.XPATH, xpath))).click()

class crawl():
    def __init__(self):
        options = Options()
        options.add_argument("window-size=1200x600")
        options.headless = True
        driver = webdriver.Chrome('/Applications/chromedriver', options=options)
        options.add_argument("enable-automation")
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-extensions")
        options.add_argument("--dns-prefetch-disable")
        options.add_argument("--disable-gpu")
        t = time.time()
        driver.set_page_load_timeout(30)
        driver.get('https://www.myhome.go.kr/hws/portal/sch/selectRentalHouseInfoListView.do')
        wait = WebDriverWait(driver,10)
        self.driver = driver
        self.wait = wait

    def rg_crawling(self, name, reg_code):
        hh_ls = []
        typeouter = []
        driver = self.driver
        wait = self.wait
        time.sleep(1)
        click_wait(wait,"//*[@id='brtcCode']/option["+reg_code+"]") # from 2 to 18  #9는 세종인데 코드수정필
        time.sleep(10)
        try:
            click_wait(wait,"//*[@id='houseTy_11']")
        except TimeoutException:
            driver.execute_script("window.stop();")
        time.sleep(10)
        click_wait(wait,"//*[@id='frm']/div[3]/span[1]/a")
        time.sleep(10)
        dict_extend(hh_ls, driver, typeouter)
        total_num = driver.find_element_by_xpath('//*[@id="totCntSpan"]/strong').text
        total_num = int(total_num.replace(',',''))-1
        next_iter = total_num//100 + 1
        if int(reg_code) in [8,9,18]: #8,9,18은 2page는 말그대로 2page가 된당!
            page_start = 2
        else:
            page_start = 4
        for num in range(0,next_iter):
            if num!=(next_iter-1):
                page_end = 13
            else:
                page_end = ((total_num % 100) // 10) + 4
            for page_num in range(page_start,page_end):
                try:
                    xpath = "//*[@id='pageDiv']/ul/li["+str(page_num)+"]/a"
                    time.sleep(3)
                    click_wait(wait,xpath)
                    time.sleep(3)
                    dict_extend(hh_ls, driver, typeouter)
                except TimeoutException:
                    driver.execute_script("window.stop();")
                    time.sleep(3)
                    dict_extend(hh_ls, driver, typeouter)
                except:
                    break
            if num!=(next_iter-1):
                time.sleep(3)
                click_wait(wait,"//*[@id='pageDiv']/ul/li[13]/a")
                time.sleep(3)
                dict_extend(hh_ls, driver, typeouter)
            else:
                break        
        
        globals()[name] = pd.DataFrame.from_dict(hh_ls)
        globals()[name+'_detail'] = pd.DataFrame(typeouter)

2=서울  
3=부산
4=대구
5=인천
6=광주
7=대전
8=울산 - 다른코드
9=세종 - 다른코드
10=경기
11=강원
12=충북
13=충남
14=전북
15=전남
16=겅북
17=경남
=충북
13=충남
14=전북
15=전남
16=겅북
17=경남
18=제주 - 다른코드

In [55]:
crawler = crawl()
crawler.rg_crawling(name='seoul',reg_code='2')
crawler.rg_crawling(name='busan',reg_code='3')
crawler.rg_crawling(name='daegu',reg_code='4')
crawler.rg_crawling(name='incheon',reg_code='5')
crawler.rg_crawling(name='gwangju',reg_code='6')
crawler.rg_crawling(name='daejeon',reg_code='7')
crawler.rg_crawling(name='ulsan',reg_code='8')
crawler.rg_crawling(name='sejong',reg_code='9')
crawler.rg_crawling(name='gyeonggi',reg_code='10')
crawler.rg_crawling(name='gangwon',reg_code='11')
crawler.rg_crawling(name='chungbuk',reg_code='12')
crawler.rg_crawling(name='chungnam',reg_code='13')
crawler.rg_crawling(name='jeonbuk',reg_code='14')
crawler.rg_crawling(name='jeonnam',reg_code='15')
crawler.rg_crawling(name='gyeongbuk',reg_code='16')
crawler.rg_crawling(name='gyeongnam',reg_code='17')
crawler.rg_crawling(name='jeju',reg_code='18')

In [60]:
def check(region, region_detail):
    region = eval(region)
    region = region.drop_duplicates()
    region_detail = eval(region_detail)
    try:
        print(region.shape[0],region_detail['2'].nunique(),
region[(region['suplyTyNm']!='매입임대')]['rnAdres'].nunique())
    except:
        print(region.shape[0],region_detail[2].nunique(),
region[(region['suplyTyNm']!='매입임대')]['rnAdres'].nunique())

In [65]:
for name in region:
    print(name, end=' ')
    check(name, name+'_detail') # 특정경우 홈페이지에서 크롤링이 되지 않았지만, 큰 문제는 아니라고 판단

seoul 840 591 597
busan 775 76 76
daegu 296 81 81
incheon 193 66 67
gwangju 736 72 72
daejeon 302 46 46
ulsan 35 24 24
sejong 23 19 19
gangwon 316 90 91
chungbuk 343 88 88
chungnam 643 95 95
jeonbuk 301 93 93
jeonnam 119 78 78
gyeongbuk 184 100 100
gyeongnam 623 100 100
jeju 49 32 32
gyeonggi 1450 483 483


In [64]:
for name in region:
    globals()[name] = pd.read_csv(name+'.csv')
    globals()[name+'_detail'] = pd.read_csv(name+'_detail.csv')

In [59]:
region = ['seoul', 'busan', 'daegu', 'incheon', 'gwangju','daejeon', 'ulsan','sejong',
'gangwon','chungbuk','chungnam','jeonbuk','jeonnam','gyeongbuk','gyeongnam','jeju', 'gyeonggi']

In [116]:
seoul.to_csv('seoul.csv',index=False,encoding='utf-8-sig') #840건, 597건
seoul_detail.to_csv('seoul_detail.csv',index=False,encoding='utf-8-sig') #591건
busan.to_csv('busan.csv',index=False,encoding='utf-8-sig') #775건, 76건
busan_detail.to_csv('busan_detail.csv',index=False,encoding='utf-8-sig') #76건
daegu.to_csv('daegu.csv',index=False,encoding='utf-8-sig') #296건, 81건
daegu_detail.to_csv('daegu_detail.csv',index=False,encoding='utf-8-sig') #81건
incheon.to_csv('incheon.csv',index=False,encoding='utf-8-sig') #193건, 67건
incheon_detail.to_csv('incheon_detail.csv',index=False,encoding='utf-8-sig') # 66건 
gwangju.to_csv('gwangju.csv',index=False,encoding='utf-8-sig') #736건, 72건
gwangju_detail.to_csv('gwangju_detail.csv',index=False,encoding='utf-8-sig') #72건 
daejeon.to_csv('daejeon.csv',index=False,encoding='utf-8-sig') #302건, 46건
daejeon_detail.to_csv('daejeon_detail.csv',index=False,encoding='utf-8-sig') # 46건
ulsan.to_csv('ulsan.csv',index=False,encoding='utf-8-sig') #35건 24건
ulsan_detail.to_csv('ulsan_detail.csv',index=False,encoding='utf-8-sig') # 24건
sejong.to_csv('sejong.csv',index=False,encoding='utf-8-sig') #23건 19건
sejong_detail.to_csv('sejong_detail.csv',index=False,encoding='utf-8-sig') # 19건
gangwon.to_csv('gangwon.csv',index=False,encoding='utf-8-sig') #316건 91건
gangwon_detail.to_csv('gangwon_detail.csv',index=False,encoding='utf-8-sig') # 90건
chungbuk.to_csv('chungbuk.csv',index=False,encoding='utf-8-sig') #343건 88건
chungbuk_detail.to_csv('chungbuk_detail.csv',index=False,encoding='utf-8-sig') # 88건 
chungnam.to_csv('chungnam.csv',index=False,encoding='utf-8-sig') #643건 95건
chungnam_detail.to_csv('chungnam_detail.csv',index=False,encoding='utf-8-sig') #95건
jeonbuk.to_csv('jeonbuk.csv',index=False,encoding='utf-8-sig') #301건 93건
jeonbuk_detail.to_csv('jeonbuk_detail.csv',index=False,encoding='utf-8-sig') #93건
jeonnam.to_csv('jeonnam.csv',index=False,encoding='utf-8-sig') #119건 78건
jeonnam_detail.to_csv('jeonnam_detail.csv',index=False,encoding='utf-8-sig') #78건 
gyeongbuk.to_csv('gyeongbuk.csv',index=False,encoding='utf-8-sig') #184건 100건
gyeongbuk_detail.to_csv('gyeongbuk_detail.csv',index=False,encoding='utf-8-sig') # 100건
gyeongnam.to_csv('gyeongnam.csv',index=False,encoding='utf-8-sig') #623건 100건
gyeongnam_detail.to_csv('gyeongnam_detail.csv',index=False,encoding='utf-8-sig')# 100건 
jeju.to_csv('jeju.csv',index=False,encoding='utf-8-sig') #49건 32건
jeju_detail.to_csv('jeju_detail.csv',index=False,encoding='utf-8-sig') #32건

In [66]:
myhome = pd.concat([seoul,busan,daegu,incheon,gwangju,daejeon,ulsan,sejong,gangwon,
chungbuk,chungnam,jeonbuk,jeonnam,gyeongbuk,gyeongnam,jeju,gyeonggi]).iloc[:,1:].drop_duplicates()

myhome_detail = pd.concat([seoul_detail, busan_detail, daegu_detail, incheon_detail,
gwangju_detail, daejeon_detail, ulsan_detail, sejong_detail, gangwon_detail, chungbuk_detail,
chungnam_detail, jeonbuk_detail, jeonnam_detail, gyeongbuk_detail,
gyeongnam_detail, jeju_detail,gyeonggi_detail]).iloc[:,1:].drop_duplicates()

In [70]:
myhome = myhome[myhome['suplyTyNm']!='매입임대']
detail_col =['임대종류','rnAdres','준공일자','hshldCo','주택유형','건물형태',
'난방방식','승강기설치여부','insttDc','공급면적(전용)','공급면적(공용)',
'임대보증금','임대료','전환보증금']
myhome_detail.columns = detail_col

In [71]:
def str_to_num(name):
    myhome_detail[name] = myhome_detail[name].str.replace(',','').str.replace('원','').str.replace('㎡','')
    try:
        myhome_detail[name] = myhome_detail[name].astype(int)  
    except:
        myhome_detail[name] = myhome_detail[name].astype(float)

In [72]:
str_to_num('공급면적(전용)')
str_to_num('공급면적(공용)')
str_to_num('임대보증금')
str_to_num('임대료')
str_to_num('전환보증금')
str_to_num('hshldCo')

In [73]:
myhome = pd.merge(myhome, myhome_detail, on=['rnAdres','hshldCo','insttDc'])
myhome.to_csv('myhome.csv', index=False)